In [1]:
from sklearn.datasets import load_breast_cancer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
cancer = load_breast_cancer(as_frame=True)
cancer.frame.sample(5).round(2)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
33,19.27,26.47,127.90,1162.0,0.09,0.17,0.17,0.08,0.19,0.06,...,30.90,161.40,1813.0,0.15,0.66,0.61,0.18,0.37,0.11,0
145,11.90,14.65,78.11,432.8,0.12,0.13,0.04,0.03,0.20,0.08,...,16.51,86.26,509.6,0.14,0.25,0.09,0.06,0.27,0.10,1
194,14.86,23.21,100.40,671.4,0.10,0.20,0.17,0.09,0.17,0.07,...,27.78,118.60,784.7,0.13,0.46,0.46,0.17,0.30,0.09,0
489,16.69,20.20,107.10,857.6,0.07,0.07,0.04,0.02,0.18,0.05,...,26.56,127.30,1084.0,0.10,0.29,0.25,0.09,0.47,0.08,0
340,14.42,16.54,94.15,641.2,0.10,0.11,0.08,0.04,0.19,0.06,...,21.51,111.40,862.1,0.13,0.34,0.38,0.14,0.31,0.09,1


In [3]:
특성별_정상필터 = {}

for 특성명 in cancer.data.columns:
    특성열 = cancer.data[특성명]
    Q1, Q3 = np.percentile(특성열, [25, 75])
    IQR = Q3 - Q1
    이상치하한 = Q1 - IQR * 1.5
    이상치상한 = Q3 + IQR * 1.5
    특성별_정상필터[특성명] = 특성열.between(이상치하한, 이상치상한)

In [4]:
정상필터 = np.logical_and.reduce(list(특성별_정상필터.values()))
print(f'전체 관측치 수: {len(정상필터):,}, 정상범위 : {np.sum(정상필터):,}')
cancer.cleaned = cancer.frame.copy()[정상필터]
cancer.cleaned.describe().round(2)

전체 관측치 수: 569, 정상범위 : 398


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
count,398.00,398.00,398.00,398.00,398.00,398.00,398.00,398.00,398.00,398.00,...,398.00,398.00,398.00,398.00,398.00,398.00,398.00,398.00,398.00,398.00
mean,13.38,18.51,86.40,570.55,0.09,0.09,0.06,0.04,0.17,0.06,...,24.72,98.64,728.47,0.13,0.21,0.21,0.10,0.28,0.08,0.75
std,2.48,3.91,16.84,222.88,0.01,0.03,0.05,0.03,0.02,0.01,...,5.64,22.13,329.65,0.02,0.11,0.15,0.05,0.04,0.01,0.43
min,6.98,9.71,43.79,143.50,0.06,0.02,0.00,0.00,0.12,0.05,...,12.02,50.41,185.20,0.08,0.03,0.00,0.00,0.18,0.06,0.00
25%,11.71,15.70,75.06,420.35,0.08,0.06,0.03,0.02,0.16,0.06,...,20.39,83.92,516.02,0.11,0.14,0.10,0.06,0.25,0.07,1.00
50%,13.00,18.18,84.06,518.30,0.09,0.08,0.05,0.03,0.17,0.06,...,24.58,93.90,632.50,0.13,0.19,0.18,0.08,0.28,0.08,1.00
75%,14.64,20.76,94.73,664.20,0.10,0.11,0.08,0.05,0.19,0.06,...,28.44,109.78,830.58,0.14,0.27,0.30,0.13,0.31,0.09,1.00
max,20.59,29.81,137.80,1320.00,0.13,0.20,0.25,0.13,0.25,0.08,...,40.54,166.40,1872.00,0.19,0.61,0.77,0.25,0.41,0.12,1.00


In [5]:
import joblib

joblib.dump(cancer, 'cancer.joblib')

cancer = joblib.load('cancer.joblib')
print(cancer.keys())
print(cancer.frame.shape, cancer.cleaned.shape)

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module', 'cleaned'])
(569, 31) (398, 31)


In [6]:
from sklearn.preprocessing import KBinsDiscretizer

구간수 = 4

Xs = pd.DataFrame(np.random.rand(12, 1))
Xs.columns = ['x1']

구간분할기 = KBinsDiscretizer(n_bins=구간수, encode='ordinal', strategy= 'uniform')
구간 = 구간분할기.fit_transform(Xs)
구간표_uniform = pd.DataFrame(구간.astype('int'), columns=['uniform'])
구간분할기 = KBinsDiscretizer(n_bins=구간수, encode='ordinal', strategy='quantile', quantile_method='linear')
구간 = 구간분할기.fit_transform(Xs)
구간표_quantile = pd.DataFrame(구간.astype('int'), columns=['quantile'])
구간분할기 = KBinsDiscretizer(n_bins=구간수, encode='onehot-dense', strategy='quantile', quantile_method='linear')
구간_원핫 = pd.DataFrame(구간분할기.fit_transform(Xs).astype('int'))

pd.concat([Xs, 구간표_uniform, 구간표_quantile, 구간_원핫], axis = 1).sort_values('x1').round(2)

,x1,uniform,quantile,0,1,2,3
11,0.07,0,0,1,0,0,0
9,0.10,0,0,1,0,0,0
2,0.26,0,0,1,0,0,0
7,0.50,1,1,0,1,0,0
6,0.53,2,1,0,1,0,0
1,0.55,2,1,0,1,0,0
4,0.60,2,2,0,0,1,0
8,0.63,2,2,0,0,1,0
5,0.64,2,2,0,0,1,0
0,0.71,2,3,0,0,0,1


In [13]:
colors = pd.Series(['R', 'G', 'B'])
정수인코딩 = colors.map({'R': 0, 'G': 1, 'B': 2})
원핫행렬 = np.eye(3).astype('int')
원핫인코딩 = pd.DataFrame(원핫행렬[정수인코딩], columns=['R', 'G', 'B'])
pd.DataFrame(원핫행렬, columns=['R', 'G', 'B'])
frame = pd.DataFrame({'색상': colors, 'ordinal': 정수인코딩})
frame = pd.concat([frame, 원핫인코딩], axis = 1)
frame

,색상,ordinal,R,G,B
0,R,0,1,0,0
1,G,1,0,1,0
2,B,2,0,0,1


In [15]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing(as_frame=True)

In [22]:
from sklearn.preprocessing import KBinsDiscretizer

구간분할기 = KBinsDiscretizer(n_bins=4, encode='ordinal', strategy='uniform')
위도경도 = housing.frame[['Longitude', 'Latitude']]
구간 = 구간분할기.fit_transform(위도경도)
pd.concat([위도경도, pd.DataFrame(구간.astype('int'), columns = ['위도구간', '경도구간'])], axis = 1)

,Longitude,Latitude,위도구간,경도구간
0,-122.23,37.88,0,2
1,-122.22,37.86,0,2
2,-122.24,37.85,0,2
3,-122.25,37.85,0,2
4,-122.25,37.85,0,2
...,...,...,...,...
20635,-121.09,39.48,1,2
20636,-121.21,39.49,1,2
20637,-121.22,39.43,1,2
20638,-121.32,39.43,1,2


In [26]:
from sklearn.model_selection import train_test_split

def 모델평가(model, data, target, **설정):
    train_data, test_data, train_target, test_target = train_test_split(data, target, **설정)
    model.fit(train_data, train_target)
    scores = {
        'train': model.score(train_data, train_target),
        'test': model.score(test_data, test_target)
    }
    return scores

In [28]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

scores = {}

knn = Pipeline([
    ('scaler', MinMaxScaler()),
    ('knn', KNeighborsRegressor())
])

튜닝기 = GridSearchCV(knn, {
    'knn__n_neighbors': range(3, 11, 2),
    'knn__p': [1, 2]
})

scores['KNN'] = 모델평가(튜닝기, housing.data, housing.target, random_state=3)
print(scores)
print(f'knn 최적 설정:\n{pd.Series(튜닝기.best_params_)}')


{'KNN': {'train': 0.7889577709917561, 'test': 0.7287236981590316}}
knn 최적 설정:
knn__n_neighbors    9
knn__p              1
dtype: int64
